In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

D:\anaconda3\envs\ML3614\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\ML3614\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\ML3614\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\ML3614\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
#This function will give the method of weight initialization
def variable_init(size):
    inputdimension = size[0]

    #Calculate the standard deviation of the normal distribution subject to a randomly generated variable
    weight_stddev = 1. / tf.sqrt(inputdimension/ 2.)
    return tf.random_normal(shape=size, stddev=weight_stddev)

In [4]:
###Define the placeholder of the input matrix, the input layer unit is 784, 
###None represents the placeholder of the batch size, and X represents the actual input picture. 
###The numeric type of the placeholder is 32-bit floating point
X = tf.placeholder(tf.float32, shape=[None, 784])

#Define the weight matrix and bias term vector of the discriminator, 
#from this we can see that the discriminant network is a three-layer fully connected network
D_W1 = tf.Variable(variable_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(variable_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

#Define the input noise of the generator as a 100-dimensional vector group,
#None is determined according to the batch size
Z = tf.placeholder(tf.float32, shape=[None, 100])

#Define the weight and bias of the generator. The input layer is 100 neurons and accepts random noise, 
#The output layer is 784 neurons, and outputs handwritten font pictures. 
#The generation network is a three-layer fully connected network according to the original paper
G_W1 = tf.Variable(variable_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(variable_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [5]:
#Define a function that can generate a random matrix of order m*n, the elements of the matrix are uniformly distributed,
#and the randomly generated z is the input of the generator
def sample_Z(m, n):
    return np.random.uniform(-1.0, 1.0, size=[m, n])

In [6]:
#define generator
def generator(z):
    
    #The first layer first calculates y=z*G_W1+G_b1, and then inputs the activation function to calculate G_h1=ReLU(y), 
#G_h1 is the output activation value of the second layer neural network
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    
    #The following two sentences calculate the activation result of the second layer propagated to the third layer. 
#The activation result of the third layer is a vector containing 784 elements. 
#This vector can be transformed into 28×28 to represent the image
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob



In [7]:
#Define the discriminator
def discriminator(x):
    
    #Calculate D_h1=ReLU(x*D_W1+D_b1), the input of this layer is a vector with 784 elements
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    
    #Calculate the output result of the third layer. Because the Sigmoid function is used, 
    #the output result is a scalar with a value between [0,1] 
    #That is to determine whether the input image is true (=1) or false (=0)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    
    #Return the true probability and the input value of the third layer. 
    #The output D_logit is to input it into tf.nn.sigmoid_cross_entropy_with_logits() to construct the loss function
    return D_prob, D_logit

In [8]:
#This function is used to output the generated picture
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [9]:

#Input random noise z and output generated samples
G_sample = generator(Z)

#Enter the real picture and the generated picture separately, and input the discriminator to judge the authenticity
D_real, D_logit_real = discriminator(X)#Get the result of the real handwritten number discriminated by the discriminator
D_fake, D_logit_fake = discriminator(G_sample)#Get the result of the generated handwritten numbers discriminated by the discriminator

#The following is the discriminator loss and generator loss of the original paper, 
#but this implementation does not use the loss function
# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# We use cross entropy as the loss function of the discriminator and generator, 
#because sigmoid_cross_entropy_with_logits internally executes the Sigmoid function on the predicted input,
#So we take the value of the last layer of the discriminator that has not been put into the activation function, that is, D_h1*D_W2+D_b2.
#tf.ones_like(D_logit_real) creates a label whose dimension is equal to D_logit_real and is a real picture.
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
#Calculate the error of the discriminator's discrimination result of the real sample (compare the result with 1)
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
#Computational error of the discrimination result of the discriminator on the false sample (that is, the handwritten number generated by the generator) (compare the result with 0)
#The loss function has two parts, namely E[log(D(x))]+E[log(1-D(G(z)))], which distinguishes true as false and false as true
D_loss = D_loss_real + D_loss_fake#Discriminator's error

#Also use cross entropy to construct the generator loss function
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
#Generator error (compare the discrimination result of the false sample returned by the discriminator with 1)

#recorde error
dreal_loss_sum = tf.summary.scalar("dreal_loss", D_loss_real) 
dfake_loss_sum = tf.summary.scalar("dfake_loss", D_loss_fake) 
d_loss_sum = tf.summary.scalar("d_loss", D_loss) 
g_loss_sum = tf.summary.scalar("g_loss", G_loss) 
 
summary_writer = tf.summary.FileWriter('snapshots/', graph=tf.get_default_graph()) #日志记录器

#Define the optimization method of the discriminator and generator as Adam algorithm, 
#the keyword var_list indicates the weight matrix updated to minimize the loss function
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
#Choose the batch size for training and the dimension of randomly generated noise
mb_size = 128
Z_dim = 100

#Read the data set MNIST and place it in the MNIST folder under the current directory data folder. 
#If there is no data at this address, download the data to this folder
mnist = input_data.read_data_sets("./data/MNIST/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
#Open a session to run the calculation graph
sess = tf.Session()

#Initialize all defined variables
sess.run(tf.global_variables_initializer())

#If the out folder does not exist in the current directory, create the folder
if not os.path.exists('out/'):
    os.makedirs('out/')

#Initialize and start iterative training, 100w times
i = 0
for it in range(1000000):
    
    #Output a picture generated by the generator every 2000 times
    if it % 2000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
    
    #next_batch extracts the next batch of pictures, this method returns a matrix, that is, shape=[mb_size, 784], 
    #each row is a picture, a total of batch size rows
    X_mb, _ = mnist.train.next_batch(mb_size)
    
    #Input data and iterate once according to the optimization method, calculate the loss and return the loss value
    _, D_loss_curr, dreal_loss_sum_value, dfake_loss_sum_value, d_loss_sum_value = sess.run([D_solver, D_loss, dreal_loss_sum, dfake_loss_sum, d_loss_sum], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr, g_loss_sum_value = sess.run([G_solver, G_loss, g_loss_sum], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it%100 ==0: #每过100次记录一下日志，可以通过tensorboard查看
        summary_writer.add_summary(dreal_loss_sum_value, it)
        summary_writer.add_summary(dfake_loss_sum_value, it)
        summary_writer.add_summary(d_loss_sum_value, it)
        summary_writer.add_summary(g_loss_sum_value, it)
        
    #every 2000 iterations, output generator loss and discriminator loss per iteration
    if it % 2000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()